In [1]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# importing stopwords for Python 3 vs Python 2.7 is different. Follow the format below to import.
#import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{'than', 'now', 've', 'needn', 'above', "shouldn't", 're', 'very', 'to', 'because', 'between', "you've", 'same', 'don', 'all', 'weren', 'by', 's', 'an', 'once', 'ain', 'do', 'not', 'mightn', 'is', 'when', 'about', 'doesn', 'was', 'shouldn', 'have', 'what', "hasn't", "won't", 'doing', 'during', 'haven', 'both', 'but', 'didn', 'so', 'if', 'some', 'then', 'each', 'most', 'couldn', 'be', 'other', 'they', "couldn't", "hadn't", 'i', "you're", 'yours', 'being', 'were', 'wouldn', 'whom', 'their', "needn't", 'with', 'again', 'how', 'does', 'hadn', 'his', 'those', 'into', 'isn', 't', 'such', 'm', 'itself', 'before', 'of', 'her', 'hers', 'am', 'the', 'down', 'and', 'until', 'them', 'll', 'himself', 'him', 'against', 'hasn', 'ma', "wouldn't", 'where', 'herself', 'we', "didn't", 'themselves', 'he', 'its', "isn't", 'after', 'off', 'up', 'below', 'out', 'a', "wasn't", 'or', "she's", "aren't", "you'll", 'theirs', 'yourself', 'over', 'did', "that'll", 'that', 'here', 'aren', "mustn't", 'our', 'having',

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.datasets import make_classification
import pickle


In [4]:
"""Function to perform battery of programmatic nlp analyses including sentiment, LSA, LDA, turn_counts, POS, WC, UNQ"""
import glob
import datetime
# getting current date and time
d = datetime.datetime.today()
path = 'C:\\Users\\das2\\Documents'
all_files = glob.glob(path + "/*.csv")
print(all_files)

['C:\\Users\\das2\\Documents\\256_VS2.opf.csv']


In [8]:
# remove the output features and keep only input features like in input csv
# 'df' holds individual csv files read
# li is the list that holds all csv files that is read by 'df'
li = []
count = 0 
for filename in all_files:
    try:
        df = pd.read_csv(filename, sep='|', index_col=None, error_bad_lines=False, encoding='UTF-8')
        df = df[['file_name', 'speaker_label', 'speech', 'speech_onset', 'speech_offset']]
        li.append(df)
    except:
        count += 1

In [9]:
li

[]

In [10]:
df

,file_name,speaker_label,speech,speech_onset,"speech_offset,,"
0,256_VS2.opf,c_speech,no.,29,"865,,"
1,256_VS2.opf,c_speech,my baby sleep.,3122,"3894,,"
2,256_VS2.opf,c_speech,my baby sleep.,4563,"6105,,"
3,256_VS2.opf,c_speech,### baby.,16778,"17703,,"
4,256_VS2.opf,c_speech,give my milk.,38065,"39273,,"
...,...,...,...,...,...
698,256_VS2.opf,p_speech,come here.,1822812,"1823744,,"
699,256_VS2.opf,p_speech,here.,1825817,"1826901,,"
700,256_VS2.opf,p_speech,come here ###,1828230,"1829469,,"
701,256_VS2.opf,p_speech,"no, D'Angelo@n.",1830297,"1831628,"


In [ ]:
df[['Dimension', 'Shape',
       'Feature_Property', 'Location_Direction', 'Amount',
       'Pattern_Comparison', 'Null']] = wb_es_ns_df[['Dimension', 'Shape',
       'Feature_Property', 'Location_Direction', 'Amount',
       'Pattern_Comparison', 'Null']].fillna(0)

In [13]:
# separating out each line and processing as list of elements
classes=[]

for row in df.iterrows():
    index, data = row
    classes.append(data.tolist())

classes=np.array(classes)

classes

array([['256_VS2.opf', 'c_speech', 'no.', '29', '865'],
       ['256_VS2.opf', 'c_speech', 'my baby sleep.', '3122', '3894'],
       ['256_VS2.opf', 'c_speech', 'my baby sleep.', '4563', '6105'],
       ...,
       ['256_VS2.opf', 'p_speech', 'come here ###', '1828230', '1829469'],
       ['256_VS2.opf', 'p_speech', "no, D'Angelo@n.", '1830297',
        '1831628'],
       ['256_VS2.opf', 'p_speech', '---', '1834601', '1837107']],
      dtype='<U141')

In [18]:
# extract speech column only for the phrases and apply for df only because li is already list
text_list = list(df['speech'])
text_list = [text for text in text_list if type(text) != int]

In [15]:
text_list

['no.',
 'my baby sleep.',
 'my baby sleep.',
 '### baby.',
 'give my milk.',
 'Ma stop.',
 'gone.',
 'this cookie.',
 'donut.',
 'huh?',
 '###',
 '###',
 'pizza.',
 '###',
 'mine.',
 'huh?',
 'chicken.',
 '###',
 "that's chip.",
 'Mommy got to like chip?',
 'I like chip.',
 'yeah.',
 'steak.',
 'yeah.',
 'my baby.',
 'thank+you.',
 'hold it.',
 'ice+cream.',
 'ice+cream.',
 'yep.',
 'yep.',
 '### mine.',
 'yep.',
 '###',
 'yeah.',
 'chocolate?',
 '###',
 'yeah.',
 'give my baby.',
 '### my baby sleep.',
 'yep.',
 'my baby -- sleep.',
 '### wake up.',
 '### gum.',
 'ok.',
 'ok.',
 '### baby.',
 'my baby.',
 'I change babber@.',
 'no wet.',
 'down+stairs.',
 'down+stairs.',
 'I read.',
 'yeah.',
 'gorilla.',
 'yeah, monkey.',
 '###',
 'bike.',
 'hmm?',
 'ball.',
 'ball.',
 'good+night elephant.',
 'good+night lion.',
 'good+night.',
 'good+night ###',
 'good+night, giraffe.',
 'good+night, armadillo.',
 'hmm?',
 '###',
 'huh?',
 '###',
 'hmm?',
 'good+night, dear.',
 'good+night.',
 '##

In [62]:

#LabelBinarizer = preprocessing.LabelBinarizer()



In [64]:
#  Sequentially apply a list of transforms and a final estimator 
# add your stop_words  (stop_words)

#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer

SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

SVC_pipeline.fit(text_list, classes)
#SVC_pipeline.fit(text_list.LabelBinarizer(), classes)


ValueError: Multioutput target data is not supported with label binarization

In [54]:
# new path
# K:\New Reorganization\Research\Research Management\Data Manager Materials\pickled_models\archived spatial models
pickle.dump(SVC_pipeline,open('C:/Users/das2/Documents/spa_SVC_063.pickle', 'wb'))

In [55]:
# training ends and model runs
# pass the english models

obj = pickle.load(open('C:/Users/das2/Documents/spa_SVC_063.pickle', 'rb'))

In [57]:
# test prediction
obj.predict(['Number', 'two', 'baby'])

NotFittedError: Vocabulary not fitted or provided

In [56]:


for df in li:
    x = obj.predict(df["speech"].tolist())
    # append to list here # check the header names
    new.append(x)

NotFittedError: Vocabulary not fitted or provided